In [1]:
import random as r
import json

from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from AbstractArchitecture_v2 import AbstractArchitecture
from DenseEncoder import DenseEncoder
from DenseDecoder import DenseDecoder
from NormalizedMeanSquaredError import NormalizedMeanSquaredError as NMSE
from plot_model_prediction import plot_model_prediction

In [2]:
# Set data file locations
data_file_prefix = './data/NLSL_expt1'

# Step 1. Load in the data
data_train_u = np.load("{}_train1_u.npy".format(data_file_prefix)).astype(np.float32)
data_train_f = np.load("{}_train1_f.npy".format(data_file_prefix)).astype(np.float32)
data_val_u = np.load("{}_val_u.npy".format(data_file_prefix)).astype(np.float32)
data_val_f = np.load("{}_val_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u1 = np.load("{}_test1_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f1 = np.load("{}_test1_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u = np.load("{}_test2_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f = np.load("{}_test2_f.npy".format(data_file_prefix)).astype(np.float32)


In [ ]:
best_model_path = 

# Load the best model
full_model = keras.models.load_model(best_model_path, 
                                     custom_objects={"NormalizedMeanSquaredError": NMSE})

# Continue training the (now full) model
checkpoint_model_path = './model_weights/{}_checkpoint_full'.format(expt_name)
cbs = [keras.callbacks.ModelCheckpoint(checkpoint_model_path,
                                       save_weights_only=True,
                                       monitor='val_loss',
                                       save_best_only=True)]
hist = full_model.fit(x=[data_train_u, data_train_f],
                      y=[data_train_u, data_train_f, data_train_f, data_train_u],
                      validation_data=val_data,
                      callbacks=cbs,
                      batch_size=batch_size,
                      epochs=final_epochs)

# Load weights with best validation loss
full_model.load_weights(checkpoint_model_path)

In [13]:
# Inputs to score model
full_model_path = 
model_weight_path = './model_weights/NLSL_Experiment_04c_checkpoint.tf'
model_weight_path = './model_weights/NLSL_Experiment_04c_best_aec_model_weights.tf'
#model_weight_path = './model_weights/NLSL_Experiment_03c_final_model_weights.tf'



In [14]:
# Reconstruct network:
activation = "relu"
initializer = tf.keras.initializers.VarianceScaling()
reg_lambda_l2 = 1e-6
regularizer = tf.keras.regularizers.l2(reg_lambda_l2)

act_layer = dict(activation=activation,
                 kernel_initializer=initializer,
                 kernel_regularizer=regularizer)
lin_layer = dict(activation=None,
                 kernel_initializer=initializer,
                 kernel_regularizer=regularizer)
latent_config = dict(activation=None,
                     kernel_regularizer=regularizer,
                     use_bias=False)

encoder_layers = 5
decoder_layers = 5
add_identity = True

# Model training setting
## Set optimizer
optimizer = keras.optimizers.Adam
optimizer_opts = {}

# Callback function(s) and fit method options
cbs = []

############################################
### Everything below here is automated!! ###
############################################


# Step 2. Set up the model architecture
_, n = data_train_u.shape

encoder_config = {'units_full': n,
                  'num_layers': encoder_layers,
                  'actlay_config': act_layer,
                  'linlay_config': lin_layer,
                  'add_init_fin': add_identity}

decoder_config = {'units_full': n,
                  'num_layers': decoder_layers,
                  'actlay_config': act_layer,
                  'linlay_config': lin_layer,
                  'add_init_fin': add_identity}

# Aggregate settings for model architecture
architecture_config = {'units_latent': l,
                       'units_full': n,
                       'u_encoder_block': DenseEncoder(**encoder_config),
                       'u_decoder_block': DenseDecoder(**decoder_config),
                       'F_encoder_block': DenseEncoder(**encoder_config),
                       'F_decoder_block': DenseDecoder(**decoder_config),
                       'latent_config': latent_config}

In [15]:
###############################################
# Step 5. Set up the full architecture run!!
###############################################

# Set up validation data, loss functions, and number of epochs
val_data = [(data_val_u, data_val_f), 
            (data_val_u, data_val_f, data_val_f, data_val_u)]
loss_fns = 4*[NMSE()]

# Instantiate the new model
full_model = AbstractArchitecture(**architecture_config,
                                  train_autoencoders_only=False)

# Load the weights
full_model.load_weights(model_weight_path)
full_model.compile(loss=loss_fns, optimizer=optimizer())


In [17]:
full_model.F_Expand

<tf.Variable 'Variable:0' shape=(128, 128) dtype=float32, numpy=
array([[ 1.4649461e-01,  1.3488597e+00,  3.4818617e-03, ...,
        -9.9592768e-02, -6.0798202e-02, -2.3643263e-01],
       [-1.7563622e-01,  4.6216230e+00, -4.3059718e-02, ...,
        -2.5134576e-02, -5.5155540e-01,  7.0619333e-01],
       [-8.2934842e-02, -1.5751172e+00,  6.3790724e-02, ...,
         3.4261156e-02,  5.2623677e-01, -2.6379162e-01],
       ...,
       [ 1.0836387e-01, -1.5195245e-01, -2.9770922e-02, ...,
         1.1947747e-01,  1.3522559e+00, -4.4808277e-01],
       [-3.4878928e-02, -9.2090733e-02,  5.6633491e-02, ...,
         1.0114634e-01,  1.9662423e+00,  3.7870198e-01],
       [ 9.8269403e-02, -4.9913174e-01,  3.4579262e-02, ...,
        -1.0193128e-01, -1.4418639e+00,  2.5331366e+00]], dtype=float32)>

In [22]:
full_model.Operator.get_operator()

<tf.Tensor: shape=(128, 128), dtype=float32, numpy=
array([[11.193986  ,  2.7592251 , -1.6591475 , ...,  0.12239839,
        -0.07826142, -0.23726544],
       [ 2.7592251 ,  7.138012  , -1.7727076 , ...,  0.151703  ,
        -0.16288912, -0.03837639],
       [-1.6591475 , -1.7727076 ,  6.351153  , ..., -0.3096179 ,
         0.08145825, -0.42786384],
       ...,
       [ 0.12239839,  0.151703  , -0.3096179 , ...,  3.5000656 ,
         0.4984079 , -0.6058025 ],
       [-0.07826142, -0.16288912,  0.08145825, ...,  0.4984079 ,
         3.4857    , -0.6894373 ],
       [-0.23726544, -0.03837639, -0.42786384, ..., -0.6058025 ,
        -0.6894373 ,  5.070664  ]], dtype=float32)>

In [5]:
## Step 2. Compute scores for training, validation, and testing datasets
#train_score = full_model.test_on_batch(x=[data_train_u, data_train_f],
#                                       y=[data_train_u, data_train_f, data_train_f, data_train_u])
#print("Training Scores:", train_score)

In [6]:
#val_score = model.test_on_batch(x=[data_val_u, data_val_f],
#                                y=[data_val_u, data_val_f, data_val_f, data_val_u])
#print("Validation Scores:", val_score)
#
#test_score_1 = model.test_on_batch(x=[data_test_u, data_test_f],
#                                   y=[data_test_u, data_test_f, data_test_f, data_test_u])
#print("Test 1 Scores:", test_score_1)
#
#test_score_2 = model.test_on_batch(x=[data_test_u1, data_test_f1],
#                                   y=[data_test_u1, data_test_f1, data_test_f1, data_test_u1])
#print("Test 2 Scores:", test_score_2)
#    
#    #return train_score, val_score, test_score_1, test_score_2

In [7]:
#scores = score_model(full_model, data_file_prefix)

In [20]:
predicted_ys = full_model.predict(x=[data_train_u, data_train_f])

In [21]:
true_ys=[data_train_u, data_train_f, data_train_f, data_train_u]
nmse = NMSE()

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, true_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i+1), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 1:  0.670635939
Score 2:  0.871644378
Score 3:  1.00046885
Score 4:  0.603007197
(8906, 128)
Total Loss: tf.Tensor(3.1457562, shape=(), dtype=float32)


In [10]:
#full_model.test_on_batch(x=[data_train_u, data_train_f],
#                         y=[data_train_u, data_train_f, data_train_f, data_train_u])

In [11]:
val_x = [data_val_u, data_val_f]
val_ys = [data_val_u, data_val_f, data_val_f, data_val_u]
predicted_ys = full_model.predict(x=val_x)

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, val_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 0:  0.000135408714
Score 1:  0.00024330795
Score 2:  0.0406286456
Score 3:  0.0626623631
(8906, 128)
Total Loss: tf.Tensor(0.103669725, shape=(), dtype=float32)
